In [1]:
# Create code to answer each of the following questions.

# Dependencies
import requests
import json

# Google API Key
from config import gkey

def google_api_request(url, params=None):
    params = params or {}
    params['key'] = gkey
    result = requests.get(url, params=params).json()
    if 'error_message' in result:
        raise Exception(result['error_message'])
    return result['results']

def geocode_results(address):
    return google_api_request('https://maps.googleapis.com/maps/api/geocode/json', params={
        'address': address,
    })

def place_results(lat, lng, radius=8000, place_type=None, keyword=None):
    params = {
        'location': f"{lat},{lng}",
        'radius': radius,
    }
    if place_type:
        params['type'] = place_type
    if keyword:
        params['keyword'] = keyword
    return google_api_request('https://maps.googleapis.com/maps/api/place/nearbysearch/json', params)

In [2]:
# 1. What are the geocoordinates (latitude and longitude) of Seattle, Washington?

seattle_geo = geocode_results('Seattle, Washington')[0]
seattle_lat = seattle_geo['geometry']['location']['lat']
seattle_lng = seattle_geo['geometry']['location']['lng']

print(f"Seattle, Washington: {seattle_lat}, {seattle_lng}")

Seattle, Washington: 47.6062095, -122.3320708


In [3]:
# 2. What are the geocoordinates (latitude and longitude) of The White House?

wh_geo = geocode_results('The White House')[0]
wh_lat = wh_geo['geometry']['location']['lat']
wh_lng = wh_geo['geometry']['location']['lng']

print(f"The White House: {wh_lat}, {wh_lng}")

The White House: 38.8976763, -77.0365298


In [4]:
# 3. Find the name and address of a bike store in Seattle, Washington.
#    Hint: See https://developers.google.com/places/web-service/supported_types

seattle_bikes = place_results(seattle_lat, seattle_lng, place_type="bicycle_store")[0]

print(seattle_bikes["name"])
print(seattle_bikes["vicinity"])

REI
222 Yale Avenue North, Seattle


In [5]:
# 4. Find a balloon store near the White House.

wh_balloons = place_results(wh_lat, wh_lng, keyword="Balloon Store")[0]

print(wh_balloons["name"])
print(wh_balloons["vicinity"])

Total Party - Party, Costumes & Novelties
1750 Crystal Dr, Arlington


In [6]:
# 5. Find the nearest dentist to your house.
# Hint: Use Google Maps to find your latitude and Google Places to find
# the dentist. You may also need the rankby property.

def place_results(lat, lng, radius=8000, place_type=None, keyword=None, rankby=None):
    params = {
        'location': f"{lat},{lng}",
        'radius': radius,
    }
    if place_type:
        params['type'] = place_type
    if keyword:
        params['keyword'] = keyword
    if rankby:
        params['rankby'] = rankby
        del params['radius']
    return google_api_request('https://maps.googleapis.com/maps/api/place/nearbysearch/json', params)

# get my geocode data
my_geo = geocode_results('Trader Joes, 92037')[0]
my_lat = my_geo['geometry']['location']['lat']
my_lng = my_geo['geometry']['location']['lng']

# find nearby dentists
my_dentist = place_results(my_lat, my_lng, place_type='dentist', rankby='distance')[0]

print(my_dentist['name'])
print(my_dentist['vicinity'])

La Jolla Village Smiles Dentistry and Implants
8657 Villa La Jolla Drive Ste 211, La Jolla


In [7]:
# 6. Bonus: Find the names and addresses of the top five restaurants in your home city.
#    Hint: Read about "Text Search Results"
# (https://developers.google.com/places/web-service/search#TextSearchRequests)

def textsearch_results(query):
    return google_api_request('https://maps.googleapis.com/maps/api/place/textsearch/json', params={
        'query': query,
    })

results = textsearch_results("best restaurant in San Diego, CA")[:5]

for num, result in enumerate(results):
    print('-- Rank {} --'.format(num + 1))
    print(result['name'])
    print(result['formatted_address'])
    print()

-- Rank 1 --
Snooze an A.M. Eatery
3435 Del Mar Heights Rd D3, San Diego, CA 92130, USA

-- Rank 2 --
Harney Sushi
3964 Harney St, San Diego, CA 92110, USA

-- Rank 3 --
CUCINA urbana
505 Laurel St, San Diego, CA 92101, USA

-- Rank 4 --
Tender Greens
2400 Historic Decatur Rd #104, San Diego, CA 92106, USA

-- Rank 5 --
Island Prime
880 Harbor Island Dr, San Diego, CA 92101, USA

